In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from pathlib import Path


In [8]:
# File to Load 
GDP_data_to_load = Path("data/GDP.csv")
particle_data_to_load = Path("data/part-con.csv")
lung_data_to_load = Path("data/dataset-cumulative-risk-in-percentage-inc-males-and-females-age-0-74-lung.csv")
melanoma_data_to_load = Path("data/dataset-cumulative-risk-in-percentage-inc-males-and-females-age-0-74-2000-2018-melanoma-of-skin.csv")

# Read Data Files and store into Pandas DataFrames
GDP_data = pd.read_csv(GDP_data_to_load)
particle_data = pd.read_csv(particle_data_to_load)
lung_data = pd.read_csv(lung_data_to_load)
melanoma_data = pd.read_csv(melanoma_data_to_load)

# Combine the data into a single DataFrame

GDP_data.head()



,Unnamed: 0,Country,Scale,Base Year,2017,2018,2019,2020,2021,2022
0,NaN,"Afghanistan, Islamic Rep. of",Millions,NaN,"1,285,460.00","1,327,690.00","1,469,600.00","1,547,290.00","1,232,858.30",...
1,NaN,Albania,Millions,NaN,"1,550,645.49","1,636,731.32","1,691,903.43","1,647,431.07","1,856,172.26","2,134,463.55"
2,NaN,Algeria,Millions,NaN,...,"20,393,524.41","20,500,200.00","18,477,000.00","22,079,300.00",...
3,NaN,"Andorra, Principality of",Millions,NaN,"2,655.76","2,725.27","2,818.42","2,531.09","2,815.42",...
4,NaN,Angola,Millions,NaN,"20,262,300.00","25,627,742.12","30,330,429.40","33,063,020.24","46,696,955.73",...


In [31]:
particle_data = pd.read_csv(particle_data_to_load)

particle_keep_columns = ["Location", "Period", "Dim1", "FactValueNumeric", "FactValueNumericLow",
                          "FactValueNumericHigh"]
particle_data = particle_data[particle_keep_columns]
particle_data = particle_data.rename(columns={"Location": "Country"})
particle_data = particle_data.rename(columns={"Dim1": "Area Type"})


particle_data.head()


,Country,Period,Area Type,FactValueNumeric,FactValueNumericLow,FactValueNumericHigh
0,Kenya,2019,Cities,10.01,6.29,13.74
1,Trinidad and Tobago,2019,Rural,10.02,7.44,12.55
2,United Kingdom of Great Britain and Northern I...,2019,Cities,10.06,9.73,10.39
3,Grenada,2019,Total,10.08,7.07,13.20
4,Brazil,2019,Towns,10.09,8.23,12.46


In [9]:
# Find the Countries that have data in all files (ex. US, Germany, Norway, etc.) and rank them from highest GDP to lower GDP?

#create a bar chart


In [10]:
# find the countries with the highest polution indices

#create a bar chart


In [11]:
# create a scatter plot with and GDP (Y) and pollution level (X)

# run a regression line

In [12]:
# Find the number of lung cancer victims for our Countries sample

# Creat a bar chart that reflects number lung cancer victims per country and their GDP and pollution level

In [13]:
# Find the number of melanoma cancer victims for our Countries sample

# Creat a bar chart

# Creat a bar chart that reflects number of melanoma victims per country and their GDP and pollution level
